## 基于MindNLP+MusicGen生成自己的个性化音乐

MusicGen是来自Meta AI的Jade Copet等人提出的基于单个语言模型（LM）的音乐生成模型，能够根据文本描述或音频提示生成高质量的音乐样本，相关研究成果参考论文《[Simple and Controllable Music Generation](https://arxiv.org/abs/2306.05284)》。

MusicGen模型基于Transformer结构，可以分解为三个不同的阶段:
1. 用户输入的文本描述作为输入传递给一个固定的文本编码器模型，以获得一系列隐形状态表示。
2. 训练MusicGen解码器来预测离散的隐形状态音频token。
3. 对这些音频token使用音频压缩模型（如EnCodec）进行解码，以恢复音频波形。

MusicGen直接使用谷歌的[t5-base](https://huggingface.co/t5-base)及其权重作为文本编码器模型，并使用[EnCodec 32kHz](https://huggingface.co/facebook/encodec_32khz)及其权重作为音频压缩模型。MusicGen解码器是一个语言模型架构，针对音乐生成任务从零开始进行训练。


MusicGen 模型的新颖之处在于音频代码的预测方式。传统上，每个码本都必须由一个单独的模型（即分层）或通过不断优化 Transformer 模型的输出（即上采样）进行预测。与传统方法不同，MusicGen采用单个stage的Transformer LM结合高效的token交织模式，取消了多层级的多个模型结构，例如分层或上采样，这使得MusicGen能够生成单声道和立体声的高质量音乐样本，同时提供更好的生成输出控制。MusicGen不仅能够生成符合文本描述的音乐，还能够通过旋律条件控制生成的音调结构。

<p align="center">
  <img src="https://github.com/sanchit-gandhi/codesnippets/blob/main/delay_pattern.png?raw=true" width="600"/>
</p>


**Figure 1:** MusicGen使用的码本延迟模式，来源于 [MusicGen paper](https://arxiv.org/abs/2306.05284).


## 下载模型

MusicGen提供了small、medium和big三种规格的[预训练权重文件](https://huggingface.co/models?search=facebook/musicgen-)，本次指南默认使用small规格的权重，生成的音频质量较低，但是生成的速度是最快的：

In [1]:
%%capture captured_output
# 实验环境已经预装了mindspore==2.2.14，如需更换mindspore版本，可更改下面mindspore的版本号
!pip uninstall mindspore -y
!pip install -i https://pypi.mirrors.ustc.edu.cn/simple mindspore==2.2.14

In [ ]:
%%capture captured_output
# 该案例在 mindnlp 0.3.1 版本完成适配，如果发现案例跑不通，可以指定mindnlp版本，执行`!pip install mindnlp==0.3.1 jieba soundfile librosa`
!pip install -i https://pypi.mirrors.ustc.edu.cn/simple mindnlp jieba soundfile librosa

In [3]:
# 查看当前 mindspore 版本
!pip show mindspore

Name: mindspore
Version: 2.2.14
Summary: MindSpore is a new open source deep learning training/inference framework that could be used for mobile, edge and cloud scenarios.
Home-page: https://www.mindspore.cn
Author: The MindSpore Authors
Author-email: contact@mindspore.cn
License: Apache 2.0
Location: /home/nginx/miniconda/envs/jupyter/lib/python3.9/site-packages
Requires: asttokens, astunparse, numpy, packaging, pillow, protobuf, psutil, scipy
Required-by: mindnlp


In [6]:
from mindnlp.transformers import MusicgenForConditionalGeneration

model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")

  0%|          | 0.00/2.20G [00:00<?, ?B/s]

  0%|          | 0.00/224 [00:00<?, ?B/s]

## 生成音乐

MusicGen支持两种生成模式：贪心（greedy）和采样（sampling）。在实际执行过程中，采样模式得到的结果要显著优于贪心模式。因此我们默认启用采样模式，并且可以在调用`MusicgenForConditionalGeneration.generate`时设置`do_sample=True`来显式指定使用采样模式。

### 无提示生成

我们可以通过方法 `MusicgenForConditionalGeneration.get_unconditional_inputs` 获得网络的随机输入，然后使用 `.generate` 方法进行自回归生成，指定 `do_sample=True` 来启用采样模式:

In [7]:
%%time
unconditional_inputs = model.get_unconditional_inputs(num_samples=1)

audio_values = model.generate(**unconditional_inputs, do_sample=True, max_new_tokens=256)

CPU times: user 7min 2s, sys: 1min 6s, total: 8min 9s
Wall time: 10min 23s


音频输出是格式是: a Torch tensor of shape `(batch_size, num_channels, sequence_length)`。

使用第三方库`scipy`将输出的音频保存为`musicgen_out.wav` 文件。

In [8]:
import scipy

sampling_rate = model.config.audio_encoder.sampling_rate
scipy.io.wavfile.write("musicgen_out.wav", rate=sampling_rate, data=audio_values[0, 0].asnumpy())

In [9]:
from IPython.display import Audio
# 要收听生成的音频样本，可以使用 Audio 在 notebook 进行播放
Audio(audio_values[0].asnumpy(), rate=sampling_rate)

参数 `max_new_tokens` 指定要生成 `token` 数。根据经验，可以使用 `EnCodec` 模型的帧速率计算出生成的音频样本的长度（以秒为单位）：

In [10]:
audio_length_in_s = 256 / model.config.audio_encoder.frame_rate

audio_length_in_s

5.12

### 文本提示生成

首先基于文本提示，通过`AutoProcessor`对输入进行预处理。然后将预处理后的输入传递给 `.generate` 方法以生成文本条件音频样本。同样，我们通过设置“do_sample=True”来启用采样模式。

其中，`guidance_scale` 用于无分类器指导（CFG），设置条件对数之间的权重（从文本提示中预测）和无条件对数（从无条件或空文本中预测）。`guidance_scale`越高表示生成的模型与输入的文本更加紧密。通过设置`guidance_scale > 1`来启用 CFG。为获得最佳效果，使用`guidance_scale=3`（默认值）生成文本提示音频。

In [22]:
%%time
from mindnlp.transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("facebook/musicgen-small")

inputs = processor(
    text=["80s pop track with bassy drums and synth", "90s rock song with loud guitars and heavy drums"],
    padding=True,
    return_tensors="ms",
)

audio_values = model.generate(**inputs, do_sample=True, guidance_scale=3, max_new_tokens=256)

  0%|          | 0.00/433 [00:00<?, ?B/s]

  0%|          | 0.00/773k [00:00<?, ?B/s]

0.00B [00:00, ?B/s]

  0%|          | 0.00/335 [00:00<?, ?B/s]

CPU times: user 6min 56s, sys: 1min 7s, total: 8min 4s
Wall time: 10min 44s


In [23]:
scipy.io.wavfile.write("musicgen_out_text.wav", rate=sampling_rate, data=audio_values[0, 0].asnumpy())

In [24]:
from IPython.display import Audio
# 要收听生成的音频样本，可以使用 Audio 在 notebook 进行播放
Audio(audio_values[0].asnumpy(), rate=sampling_rate)

### 音频提示生成

`AutoProcessor`同样可以对用于音频预测的音频提示进行预处理。在以下示例中，我们首先加载音频文件，然后进行预处理，并将输入给到网络模型来进行音频生成。最后，我们将生成出来的音频文件保存为` musicgen_out_audio.wav `

In [27]:
%%time
from datasets import load_dataset

processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
dataset = load_dataset("sanchit-gandhi/gtzan", split="train", streaming=True)
sample = next(iter(dataset))["audio"]

# take the first half of the audio sample
sample["array"] = sample["array"][: len(sample["array"]) // 2]

inputs = processor(
    audio=sample["array"],
    sampling_rate=sample["sampling_rate"],
    text=["80s blues track with groovy saxophone"],
    padding=True,
    return_tensors="ms",
)

audio_values = model.generate(**inputs, do_sample=True, guidance_scale=3, max_new_tokens=256)

CPU times: user 7min 9s, sys: 1min 7s, total: 8min 17s
Wall time: 14min 47s


In [28]:
scipy.io.wavfile.write("musicgen_out_audio.wav", rate=sampling_rate, data=audio_values[0, 0].asnumpy())

In [29]:
from IPython.display import Audio
# 要收听生成的音频样本，可以使用 Audio 在 notebook 进行播放
Audio(audio_values[0].asnumpy(), rate=sampling_rate)

为了演示批量音频提示生成，我们将按两个不同的比例对样本音频进行切片，以提供两个不同长度的音频样本。由于输入音频提示的长度各不相同，因此在传递到模型之前，它们将被填充到批处理中最长的音频样本的长度。

要恢复最终音频样本，可以对生成的audio_values进行后处理，以再次使用处理器类删除填充：

In [30]:
sample = next(iter(dataset))["audio"]

# take the first quater of the audio sample
sample_1 = sample["array"][: len(sample["array"]) // 4]

# take the first half of the audio sample
sample_2 = sample["array"][: len(sample["array"]) // 2]

inputs = processor(
    audio=[sample_1, sample_2],
    sampling_rate=sample["sampling_rate"],
    text=["80s blues track with groovy saxophone", "90s rock song with loud guitars and heavy drums"],
    padding=True,
    return_tensors="ms",
)

audio_values = model.generate(**inputs, do_sample=True, guidance_scale=3, max_new_tokens=256)

# post-process to remove padding from the batched audio
audio_values = processor.batch_decode(audio_values, padding_mask=inputs.padding_mask)

'(ReadTimeoutError("HTTPSConnectionPool(host='hf-mirror.com', port=443): Read timed out. (read timeout=10)"), '(Request ID: fc36c098-ca3d-44ca-92a7-fe8b3aeb1738)')' thrown while requesting GET https://hf-mirror.com/datasets/sanchit-gandhi/gtzan/resolve/4bd857132cb0e731bef3ec68558e7acc0a85f144/data/train-00000-of-00003-abaaa5719027ce5c.parquet
Retrying in 1s [Retry 1/5].


In [31]:
Audio(audio_values[0], rate=sampling_rate)

# 生成配置
控制生成过程的默认参数（例如采样、指导比例和生成的令牌数量）可以在模型的生成配置中找到，并根据需要进行更新。首先，我们检查默认的生成配置：

In [32]:
model.generation_config

In [33]:
# increase the guidance scale to 4.0
model.generation_config.guidance_scale = 4.0

# set the max new tokens to 256
model.generation_config.max_new_tokens = 256

# set the softmax sampling temperature to 1.5
model.generation_config.temperature = 1.5

现在重新运行生成将使用生成配置中新定义的值

In [34]:
audio_values = model.generate(**inputs)

请注意，传递给 generate 方法的任何参数都将取代生成配置中的参数，因此在调用 generate 中设置 do_sample=False 将取代生成配置中 model.generation_config.do_sample 的设置。